In [ ]:
import httplib2
from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
import os
from dotenv import load_dotenv
project_folder = os.path.expanduser('/Users/oleksandrdagayev/code/jupyter_SEO/')  # adjust as appropriate
load_dotenv(os.path.join(project_folder, '.env'))
import numpy as np
import pandas as pd
import json
from collections import defaultdict as dd

In [ ]:
# Redirect URI for installed apps
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'

# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print('Go to the following link in your browser: ' + authorize_url)

In [ ]:
#the above gets you to the credential setp.

In [ ]:
code = "4/vgEKALaNUYUIFroSs1Q_t1BswxlNcDk4OozSGOOmRB4Y3J76lVe_wUw"
credentials = flow.step2_exchange(code)

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)

In [ ]:
webmaster_search_analytics = build('webmasters', 'v3', http=http) #build the service

In [ ]:
targetdomain = '' #specify your domain here. Make sure to include protocol and if domain is www. or not. 

In [ ]:
request = {
    'startDate': '2020-01-11', #all requests must include a start AND end date
    'endDate': '2020-01-12',
    'dimensions': ['date','query','page'], #select the dimensions of your response
    'rowLimit':25000,
    'startRow':0
      }

In [ ]:
test_response = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()

In [ ]:
test_data = {'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]}
    
for i in test_response['rows']:
    date = i['keys'][0] #assumes that date is first
    data['date'].append(date)
    searchQuery = i['keys'][1] #assumes that query is second
    data['searchQuery'].append(searchQuery)
    page = i['keys'][2] #assumes that query is second
    data['page'].append(page)
    clicks = i['clicks']
    data['clicks'].append(clicks)
    impressions = i['impressions']
    data['impressions'].append(impressions)
    ctr = i['ctr']
    data['ctr'].append(ctr)
    position = i['position']
    data['position'].append(position)

In [ ]:
pd.DataFrame(test_data)

In [ ]:
def get_all_response(domain,request):
    data = {'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]}
    startRow = 0
    request['startRow'] = startRow
    initresponse = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()
    last_call_len = len(initresponse['rows'])
    mapped_dictionary = map_response_to_dict(initresponse, data)
    while last_call_len == 25000:
        print('this is running')
        startRow = startRow + 25000
        request['startRow'] = startRow
        response = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()
        last_call_len = len(response['rows'])
        mapped_dictionary = map_response_to_dict(response, data)
    return mapped_dictionary
        

In [ ]:
def map_response_to_dict(response, data):
    for i in response['rows']:
        date = i['keys'][0] #assumes that date is first
        data['date'].append(date)
        searchQuery = i['keys'][1] #assumes that query is second
        data['searchQuery'].append(searchQuery)
        page = i['keys'][2] #assumes that page is third
        data['page'].append(page)
        clicks = i['clicks']
        data['clicks'].append(clicks)
        impressions = i['impressions']
        data['impressions'].append(impressions)
        ctr = i['ctr']
        data['ctr'].append(ctr)
        position = i['position']
        data['position'].append(position)
    return data

In [ ]:
big_dict = get_all_response(targetdomain,request)

In [ ]:
pd.DataFrame(big_dict)